In [1]:
import random
import numpy as np
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, Flatten
import torch

# Load the intents JSON
intents = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": [
                "Hi",
                "Hello",
                "Hey",
                "Is anyone there?",
                "Good morning",
                "Good evening",
                "Greetings",
                "Hey there"
            ],
            "responses": [
                "Hello! How can we assist you today?",
                "Hi there! Welcome to Supermexx, your battery solution provider.",
                "Greetings from Supermexx! How can we help you today?"
            ]
        },
        {
            "tag": "goodbye",
            "patterns": [
                "Bye",
                "Goodbye",
                "See you later",
                "Talk to you soon",
                "Farewell",
                "Catch you later"
            ],
            "responses": [
                "Goodbye! We hope to hear from you soon.",
                "Take care! Reach out if you have more questions about Supermexx batteries.",
                "Thank you for visiting Supermexx!"
            ]
        },
        {
            "tag": "thanks",
            "patterns": [
                "Thanks",
                "Thank you",
                "That's helpful",
                "Thanks for the help",
                "Much appreciated"
            ],
            "responses": [
                "You're welcome! We're happy to assist you.",
                "My pleasure! Let us know if you need further help.",
                "Glad to help!"
            ]
        },
        {
            "tag": "createaccount",
            "patterns": [
                "I need to create an account",
                "How do I open an account?",
                "I want to create a new account",
                "Can you help me sign up?",
                "How to register for an account"
            ],
            "responses": [
                "Visit our website at www.supermexx.com to create an account easily.",
                "Please follow the instructions on our site to create your Supermexx account.",
                "We offer a seamless account creation process on our homepage at Supermexx."
            ]
        },
        {
            "tag": "product_inquiry",
            "patterns": [
                "Tell me about your batteries for inverters",
                "What kind of batteries do you offer for e-bikes?",
                "What are the specs of your batteries for solar?",
                "I need details about your battery types for rickshaws",
                "What are your battery capacities for solar applications?"
            ],
            "responses": [
                "We offer high-quality lithium-ion batteries for inverters, e-bikes, rickshaws, and solar systems. Visit www.supermexx.com for more details.",
                "Our batteries range from small to high-capacity applications, including inverters, e-bikes, rickshaws, and solar solutions.",
                "We have batteries for a variety of uses, including solar systems, inverters, and electric vehicles like e-bikes and rickshaws."
            ]
        },
        {
            "tag": "battery_safety",
            "patterns": [
                "Are your batteries safe?",
                "What are the safety measures for your batteries?",
                "Can your batteries overheat?",
                "Are lithium-ion batteries dangerous?",
                "What precautions should I take with your batteries?"
            ],
            "responses": [
                "Our lithium-ion batteries for inverters, e-bikes, rickshaws, and solar applications are designed with safety in mind. Please refer to our safety guide for best practices.",
                "Yes, our batteries meet safety standards and certifications. We prioritize safety in all our products, including those for solar and e-bike systems.",
                "Our batteries are engineered with multiple layers of protection to ensure safe use in various applications."
            ]
        },
        {
            "tag": "shipping_inquiry",
            "patterns": [
                "Do you ship internationally?",
                "What are your shipping options?",
                "How long does delivery take?",
                "Can I track my order?",
                "Do you offer express shipping?"
            ],
            "responses": [
                "Yes, Supermexx ships internationally. Shipping times may vary depending on the location.",
                "You can track your order using the tracking number sent via email. Visit www.supermexx.com/shipping for more details.",
                "We offer various shipping options, including express delivery for faster service."
            ]
        },
        {
            "tag": "pricing_inquiry",
            "patterns": [
                "How much does your battery for an e-bike cost?",
                "What is the price of a 2000mAh inverter battery?",
                "Are there discounts for bulk orders?",
                "What are your pricing options for rickshaw batteries?",
                "Do you offer any discounts on solar batteries?"
            ],
            "responses": [
                "Our prices vary by battery type and capacity. Visit www.supermexx.com/pricing for detailed information on our products.",
                "We offer competitive pricing on all of our lithium-ion batteries, including for e-bikes, inverters, rickshaws, and solar applications.",
                "Yes, we offer bulk discounts on large orders. Please contact our sales team for more information."
            ]
        },
        {
            "tag": "recycling_inquiry",
            "patterns": [
                "How do I recycle lithium-ion batteries?",
                "Do you offer recycling services?",
                "Where can I dispose of old batteries?",
                "What is your battery recycling program?",
                "Can I send old batteries to you for recycling?"
            ],
            "responses": [
                "Supermexx offers a battery recycling program. Please visit our recycling page at www.supermexx.com/recycling for details.",
                "You can drop off old batteries at authorized recycling centers listed on our website. We prioritize responsible disposal.",
                "For lithium-ion battery recycling, please contact our support team, and we will guide you on the proper process."
            ]
        },
        {
            "tag": "technical_support",
            "patterns": [
                "My battery is not working",
                "I need help with my battery",
                "Can you assist with a battery issue?",
                "How do I troubleshoot a battery problem?",
                "Battery not charging properly"
            ],
            "responses": [
                "Please visit our support page at www.supermexx.com/support for troubleshooting steps and resources.",
                "Our support team is here to help with any issues you are facing with our batteries. Let us know the details, and we can guide you through the process.",
                "Ensure that you're using the correct charger and check the connections. Visit our site for further troubleshooting."
            ]
        },
        {
            "tag": "bulk_orders",
            "patterns": [
                "Do you accept bulk orders?",
                "Can I place a large order?",
                "What is the minimum quantity for bulk orders?",
                "Are there bulk discounts available?",
                "Can I get a quote for a bulk order?"
            ],
            "responses": [
                "Yes, we accept bulk orders. Please contact our sales team at sales@supermexx.com for a custom quote.",
                "We offer competitive pricing for bulk purchases on all battery types. Contact us for more details.",
                "You can place bulk orders for inverter, e-bike, rickshaw, and solar batteries through our website or sales team."
            ]
        }
    ]
}

c:\Users\matha\Project_ChatBot_LithiumBattery\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Prepare the data for the model
patterns = []
labels = []
classes = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        labels.append(intent['tag'])
    if intent['tag'] not in classes:
        classes.append(intent['tag'])

In [3]:
# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

In [4]:
# Tokenize and vectorize the text patterns
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(patterns)
word_index = tokenizer.word_index

In [5]:
# Convert patterns into sequences of integers
X = tokenizer.texts_to_sequences(patterns)
X = pad_sequences(X, padding='post')

In [6]:
# Define the deep learning model for intent classification
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=16, input_length=X.shape[1]))
model.add(LSTM(16, return_sequences=True))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(len(classes), activation='softmax'))

c:\Users\matha\Project_ChatBot_LithiumBattery\env\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [7]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - accuracy: 0.0773 - loss: 2.3994
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.1095 - loss: 2.3963 
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.1312 - loss: 2.3945 
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.1208 - loss: 2.3923 
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.1434 - loss: 2.3902 
Epoch 6/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.2076 - loss: 2.3868 
Epoch 7/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2094 - loss: 2.3851
Epoch 8/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.2415 - loss: 2.3807
Epoch 9/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.2311 - loss: 2.3770 
Epoch 10/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.2302 - loss: 2.3729 
Epoch 11/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2406 - loss: 2.3670 
Epoch 12/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.19

In [8]:
# Load the pretrained model from Hugging Face (DialoGPT)
pretrained_model_name = "microsoft/DialoGPT-small"
gpt_tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name)
gpt_model = GPT2LMHeadModel.from_pretrained(pretrained_model_name)

c:\Users\matha\Project_ChatBot_LithiumBattery\env\Lib\site-packages\transformers\modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.loa

In [9]:
# Function to predict the intent of a given user input
def predict_intent(text):
    seq = tokenizer.texts_to_sequences([text])
    padded_seq = pad_sequences(seq, maxlen=X.shape[1], padding='post')
    pred = model.predict(padded_seq)
    return classes[np.argmax(pred)]

In [10]:
# Function to get a random response based on predicted intent
def get_response(intent):
    for i in intents['intents']:
        if i['tag'] == intent:
            return random.choice(i['responses'])

In [11]:
# Function to generate a fallback response using Hugging Face's DialoGPT
def generate_fallback_response(user_input):
    inputs = gpt_tokenizer.encode(user_input + gpt_tokenizer.eos_token, return_tensors="pt")
    outputs = gpt_model.generate(inputs, max_length=100, pad_token_id=gpt_tokenizer.eos_token_id)
    response = gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [12]:
# Chatbot Loop
while True:
    user_input = input("You: ")
    predicted_intent = predict_intent(user_input)
    
    # If the predicted intent matches one in the predefined intents, use it
    if predicted_intent in classes:
        response = get_response(predicted_intent)
    else:
        # If no intent is found, use the fallback model
        response = generate_fallback_response(user_input)
    
    print("Bot:", response)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step
Bot: For lithium-ion battery recycling, please contact our support team, and we will guide you on the proper process.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Bot: Please visit our support page at www.supermexx.com/support for troubleshooting steps and resources.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Bot: Please visit our support page at www.supermexx.com/support for troubleshooting steps and resources.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Bot: You're welcome! We're happy to assist you.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Bot: Glad to help!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Bot: My pleasure! Let us know if you need further help.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Bot: Glad to help!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Bot: My pleasure! Let us know if you need further help.
